### Predictive Analytics for 311 Service Request Resolution | Data Pre-processing Notebook

---
*This notebook explores data from all cities listed in Andew Friedman's 311 Dataset repository.* **https://andrew-friedman.github.io/jkan/datasets/**

In [92]:
### Example Calls for the 15 Cities with Useful 311 Data
#                input_path,                  output_path,                         city_name,       date_open_col,              date_closed_col,           department_col
##################################################################################################################################################################################################################
#clean_city_data('../data/baltimore.csv',     '../data/baltimore_cleaned.csv',     'baltimore',     'CreatedDate',              'CloseDate',               'Agency')    
#clean_city_data('../data/boston.csv',        '../data/boston_cleaned.csv',        'boston',        'open_dt',                  'closed_dt',               'subject')          
#clean_city_data('../data/buffalo.csv',       '../data/buffalo_cleaned.csv',       'buffalo',       'Open Date',                'Closed Date',             'Subject')        
#clean_city_data('../data/dallas.csv',        '../data/dallas_cleaned.csv',        'dallas',        'Created Date',             'Closed Date',             'Department')                                                      
#clean_city_data('../data/denver.csv',        '../data/denver_cleaned.csv',        'denver',        'Case Created dttm',        'Case Closed dttm',        'Agency')            
#clean_city_data('../data/los_angeles.csv',   '../data/los_angeles_cleaned.csv',   'los_angeles',   'CreatedDate',              'ClosedDate',              'AssignTo')       
#clean_city_data('../data/louisville.csv',    '../data/louisville_cleaned.csv',    'louisville',    'requested_datetime',       'closed_date',             'agency_responsible') 
#clean_city_data('../data/memphis.csv',       '../data/memphis_cleaned.csv',       'memphis',       'CREATION_DATE',            'CLOSE_DATE',              'DEPARTMENT')                        
#clean_city_data('../data/in/miami_2.csv',         '../data/in/miami_cleaned_2.csv',         'miami',         'ticket_created_date_time', 'ticket_closed_date_time', 'case_owner_description')
#clean_city_data('../data/minneapolis.csv',   '../data/minneapolis_cleaned.csv',   'minneapolis',   'OPENEDDATETIME',           'CLOSEDDATETIME',          'SUBJECTNAME')
#clean_city_data('../data/nashville.csv',     '../data/nashville_cleaned.csv',     'nashville',     'Date / Time Opened',       'Date / Time Closed',      'Request Type')
#clean_city_data('../data/oakland.csv',       '../data/oakland_cleaned.csv',       'oakland',       'DATETIMEINIT',             'DATETIMECLOSED',          'REQCATEGORY')
#clean_city_data('../data/philadelphia.csv',  '../data/philadelphia_cleaned.csv',  'philadelphia',  'requested_datetime',       'closed_datetime',         'agency_responsible')
#clean_city_data('../data/san_antonio.csv',   '../data/san_antonio_cleaned.csv',   'san_antonio',   'date_requested',           'date_closed',             'Dept')
#clean_city_data('../data/washington_dc.csv', '../data/washington_dc_cleaned.csv', 'washington_dc', 'INITIATEDDATE',            'CLOSEDDATE',              'REQUESTCATEGORY')    

In [90]:
import pandas as pd

def clean_city_data(input_path, output_path, city_name, date_open_col, date_closed_col, department_col):
    """
    Clean city data from a CSV file, filtering by date range, and sorting by open date.
    As each city's 311 datasets are organized differently, we can use this function to clean data by city.

    Args:
        input_path (str): Path to the input CSV file.
        output_path (str): Path to save the cleaned CSV file.
        city_name (str): Name of the city.
        date_open_col (str): Name of the column containing open date.
        date_closed_col (str): Name of the column containing closed date.
        department_col (str): Name of the column containing department information.

    Outputs:
        Cleaned DataFrame as a CSV named "city_cleaned.csv"
    
    Returns:
        pandas.DataFrame: Head of the cleaned DataFrame.
    """
    try:
        # Try loading data with default encoding
        try:
            df = pd.read_csv(input_path)
        except UnicodeDecodeError:
            # If default encoding fails, try loading with 'cp1252' encoding
            df = pd.read_csv(input_path, encoding='cp1252')

        # Streamline Data Transformation
        df = df[[date_open_col, date_closed_col, department_col]]
        df.rename(columns={date_open_col: 'open_date', date_closed_col: 'close_date', department_col: 'department'}, inplace=True)
        df['open_date'] = pd.to_datetime(df['open_date'], errors='coerce')  # Specify errors='coerce' to handle unconvertible data
        df['close_date'] = pd.to_datetime(df['close_date'], errors='coerce') # Specify errors='coerce' to handle unconvertible data

        # Handling Missing Data
        df.dropna(subset=['open_date', 'close_date', 'department'], inplace=True)

        # Filter data by date range (2014-01-01 to 2023-12-31)
        start_date = pd.Timestamp("2014-01-01")
        end_date = pd.Timestamp("2023-12-31")
        df = df[(df['open_date'] >= start_date) & (df['open_date'] <= end_date)]

        # Add constant column for city name
        df['city'] = city_name

        # Sort by open_date
        df.sort_values(by='open_date', inplace=True)

        # Save Cleaned DataFrame
        df.to_csv(output_path, index=False)
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return df.head(5)


In [88]:
### This is what the data looks like when merged...

#   open_date	          close_date	          department	                    city
#1	2024-01-02 17:14:00	  2024-01-04 06:34:00	 Dept of Public Works	            buffalo
#2	2024-01-02 17:19:00	  2024-01-08 10:23:00	 Dept of Public Works	            buffalo
#3	2024-01-02 18:43:00	  2024-01-03 13:16:00	 Utilities	                        buffalo
#4	2024-01-02 18:44:00	  2024-01-03 13:57:00    Dept of Public Works	            buffalo
#5	2024-01-02 19:00:00	  2024-01-03 09:12:00	 Buffalo Police Department	        buffalo
#6	2022-12-31 13:34:29	  2023-01-05 07:19:50	 Public Works	                    denver
#7	2022-12-31 13:16:48	  2023-01-03 13:32:26    Community Planning & Development	denver
#8	2022-12-31 11:24:47	  2023-01-04 10:27:17	 Community Planning & Development	denver
#9	2022-12-31 08:16:22	  2023-01-06 10:50:44	 Community Planning & Development	denver
#10	2022-12-31 11:21:00	  2023-01-20 14:36:24	 Community Planning & Development	denver

<style>
    .heatMap {
        width: 70%;
        text-align: center;
    }
    .heatMap th {
        background: black;
        word-wrap: break-word;
        text-align: center;
    }
    .heatMap tr:nth-child(1) { background: red; }
    .heatMap tr:nth-child(2) { background: green; }
    .heatMap tr:nth-child(3) { background: green; }
    .heatMap tr:nth-child(4) { background: green; }
    .heatMap tr:nth-child(5) { background: red; }
    .heatMap tr:nth-child(6) { background: green; }
    .heatMap tr:nth-child(7) { background: green; }
    .heatMap tr:nth-child(8) { background: red; }
    .heatMap tr:nth-child(9) { background: red; }
    .heatMap tr:nth-child(10) { background: red; }
    .heatMap tr:nth-child(11) { background: green; }
    .heatMap tr:nth-child(12) { background: green; }
    .heatMap tr:nth-child(13) { background: green; }
    .heatMap tr:nth-child(14) { background: green; }
    .heatMap tr:nth-child(15) { background: red; }
    .heatMap tr:nth-child(16) { background: green; }
    .heatMap tr:nth-child(17) { background: green; }
    .heatMap tr:nth-child(18) { background: red; }
    .heatMap tr:nth-child(19) { background: green; }
    .heatMap tr:nth-child(20) { background: green; }
    .heatMap tr:nth-child(21) { background: red; }
    .heatMap tr:nth-child(22) { background: red; }
    .heatMap tr:nth-child(23) { background: green; }
    .heatMap tr:nth-child(24) { background: red; }
    .heatMap tr:nth-child(25) { background: red; }
    .heatMap tr:nth-child(26) { background: green; }
</style>

<div class="heatMap">

| City            | Compatible | Avalibility       | Notes / Errors      | Website                                                                                                      |
| :-------------: | :--------: | :---------------: | :-------------------------------: | :----------------------------------------------------------------------------------------------------------: |
| Austin, TX           | NO         | N/A               | Department Column N/A             | https://data.austintexas.gov/Utilities-and-City-Services/Austin-311-Public-Data/xwdj-i9he/about_data         |
| Baltimore, MD        | YES        | 2003 - Current    | N/A                               | https://data.baltimorecity.gov/City-Services/311-Customer-Service-Requests/9agw-sxsr                         |
| Boston, MA           | YES        | 2011 - 2024       | N/A                               | https://data.boston.gov/dataset/311-service-requests                                                         |
| Buffalo, NY          | YES        | JUL2008 - Current | N/A                               | https://data.buffalony.gov/Quality-of-Life/311-Service-Requests/whkc-e5vr/about_data                         |
| Chicago, IL          | NO         | N/A               | Export Server Error               | https://data.cityofchicago.org/Service-Requests/311-Service-Requests/v6vf-nfxy/about_data                    |
| Dallas, TX           | YES        | OCT2019 - SEP2020 | N/A                               | https://www.dallasopendata.com/Services/311-Service-Requests-October-1-2019-to-September-3/m36q-vtbr/explore |
| Denver, CO           | YES        | 2007 - Current    | N/A                               | https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-311-service-requests-2007-to-current    |
| Houston, TX          | NO         | N/A               | Could Not Find Data               | https://andrew-friedman.github.io/jkan/datasets/311-City-of-Houston/                                         |
| Kansas City, MO      | NO         | N/A               | Export Server Error               | https://data.kcmo.org/311/311-Call-Center-Service-Requests-2007-March-2021/7at3-sxhp/about_data              |
| Las Vegas, CA        | NO         | N/A               | Could Not Find Data               | https://andrew-friedman.github.io/jkan/datasets/311-City-of-Las-Vegas/                                       |
| Los Angeles, CA      | YES        | 2019              | Coded departments                 | https://data.lacity.org/City-Infrastructure-Service-Requests/MyLA311-Service-Request-Data-2019/pvft-t768/about_data   |
| Louisville, KY       | YES        | 1997-Current      | Only Date, No Time                | https://data.louisvilleky.gov/search?tags=311%2520services                                                   |
| Memphis, TN          | YES        | 2016-Current      | N/A                               | https://data.memphistn.gov/dataset/Service-Requests-since-2016/hmd4-ddta/about_data                          |
| Miami, FL            | YES        | 2014-Current      | N/A                               | https://gis-mdc.opendata.arcgis.com/datasets/fce9527342684373adf6c52aa0cd1932_0/about                        |
| Milwaukee, WI        | NO         | N/A               | Department Column N/A             | https://data.milwaukee.gov/dataset/callcenterdatacurrent                                                     |
| Minneapolis, MN      | YES        | 2016-Current      | N/A                               | https://opendata.minneapolismn.gov/search?tags=311                                                           |
| Nashville, TN        | YES        | 2017-Current      | N/A                               | https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data            |
| New Orleans, LA      | NO         | 2012-2018         | Department Column N/A             | https://data.nola.gov/City-Administration/311-Calls-Historic-Data-2012-2018-/3iz8-nghx/about_data            |
| Oakland, CA          | YES        | 2012-Current      | N/A                               | https://data.oaklandca.gov/Infrastructure/OAK-311-Service-Request-Map/yp8e-dukj                              |
| Philadelphia, PA     | YES        | 2015-Current      | N/A                               | https://opendataphilly.org/datasets/311-service-and-information-requests/                                    |
| Phoenix, AZ          | NO         | 2015-Current      | Department and Closed Columns N/A | https://www.phoenixopendata.com/dataset/calls-for-service                                                    |
| Sacramento, CA       | NO         | 2016-Current      | Closed Column N/A                 | https://data.cityofsacramento.org/search?tags=Service%2520Requests                                           |
| San Antonio, TX      | YES        | For The Last Year | Only Date, No Time                | https://data.sanantonio.gov/dataset/service-calls                                                            |
| San Diego, CA        | NO         | 2016-Current      | Department Column N/A             | https://data.sandiego.gov/datasets/get-it-done-311/                                                          |
| San Francisco, CA    | NO         | 2008-Current      | Export Server Error               | https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6/about_data                                    |
| Washington, D.C.     | YES        | 2009-Current      | Coded departments                               | https://opendata.dc.gov/datasets/cityworks-service-requests/explore                                          |

</div>